In [1]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
# Folder path 
folder_path = 'D:\FPTUni\SP24\ADY201m\Lab05\ADY201m_Lab05_SE183256'
# Import the dataset
train_data = pd.read_csv(os.path.join(folder_path, 'train.csv'))
test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'))
output_path = os.path.join(folder_path, 'submission.csv')

In [3]:
# Save ID of test data
test_id = test_data['Id']

In [4]:
# Drop unnecessary columns
train_data = train_data.drop('Id', axis=1)

In [5]:
# Setup threshold for missing values

# === IMPORTANT ===


# Here we will put the variable to choose the threshold
threshold = 0.5


# === IMPORTANT ===

In [6]:
# Because MSSubClass is int64 but it is a categorical variable, so we need to convert it to string
train_data['MSSubClass'] = train_data['MSSubClass'].astype(str)

In [7]:
# Get the categorical columns
categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()
# Delete all categorical columns
train_data = train_data.drop(categorical_cols, axis=1)

In [8]:
# Compute the correlation matrix with SalePrice
corr_matrix = train_data.corr()
corr_matrix['SalePrice'].sort_values(ascending=False)
# ABS of correlation coefficient
# Get the features that have the correlation coefficient with SalePrice greater than the threshold
selected_features = corr_matrix['SalePrice'][corr_matrix['SalePrice'].abs() > threshold].index.tolist()
# Display the selected features

In [9]:
# Create the scaler
scaler = StandardScaler()
scaler2 = StandardScaler()
# Fit the scaler to the train data
scaler.fit(train_data[selected_features])
# Transform the train data
train_data_scaled = scaler.transform(train_data[selected_features])

In [10]:
# Convert test data to same format with train data to predict, ( test data doest not have SalePrice column)
# remove saleprice column from selected_features and make new selected_features2
selected_features2 = selected_features.copy()
selected_features2.remove('SalePrice')
# if Id have in test data, we need to remove it, if not, we don't need to remove it
if 'Id' in test_data.columns:
    test_data = test_data.drop('Id', axis=1)
scaler2.fit(test_data[selected_features2])
test_data_scaled = scaler2.transform(test_data[selected_features2])

In [11]:
# Prepare the data
y_train = train_data_scaled[:, -1]
x_train = train_data_scaled[:, :-1]

# Split the train data into train and test data
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [12]:
# Create the model
model01 = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(len(selected_features) - 1,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='linear')
])

# Compile the model
model01.compile(optimizer='adam', loss='mse', metrics=['mae'])
# Train the model
model01.fit(x_train, y_train, epochs=8, batch_size=16, validation_data=(x_test, y_test))



Epoch 1/8


73/73 [==============================] - 1s 6ms/step - loss: 0.3782 - mae: 0.3913 - val_loss: 0.2837 - val_mae: 0.2993
Epoch 2/8
73/73 [==============================] - 0s 3ms/step - loss: 0.2234 - mae: 0.2873 - val_loss: 0.2336 - val_mae: 0.2785
Epoch 3/8
73/73 [==============================] - 0s 3ms/step - loss: 0.1922 - mae: 0.2784 - val_loss: 0.1923 - val_mae: 0.2780
Epoch 4/8
73/73 [==============================] - 0s 3ms/step - loss: 0.1888 - mae: 0.2719 - val_loss: 0.1894 - val_mae: 0.2704
Epoch 5/8
73/73 [==============================] - 0s 2ms/step - loss: 0.1713 - mae: 0.2624 - val_loss: 0.1968 - val_mae: 0.2643
Epoch 6/8
73/73 [==============================] - 0s 2ms/step - loss: 0.1677 - mae: 0.2576 - val_loss: 0.1739 - val_mae: 0.2521
Epoch 7/8
73/73 [==============================] - 0s 2ms/step - loss: 0.1543 - mae: 0.2494 - val_loss: 0.1780 - val_mae: 0.2492
Epoch 8/8
73/73 [==============================] - 0s 2ms/step - loss: 0.1522 - mae: 0.2435 -

In [14]:
# Predict using the trained model
result_scaled = model01.predict(test_data_scaled)

# Combine scaled test data with scaled result
combined_data_scaled = np.concatenate((test_data_scaled, result_scaled), axis=1)
# Unscale the combined data to get the final result
result = scaler.inverse_transform(combined_data_scaled)[:, -1]

# Kiểm tra result có cái nào trống hoặc là số âm không, nếu có thì thay nó bằng 0
result = np.where(result < 0, 0, result)
result = np.where(np.isnan(result), 0, result)

46/46 [==============================] - 0s 1ms/step


In [15]:
# Create the submission file
submission = pd.DataFrame({'Id': test_id, 'SalePrice': result})
# Save the submission file
submission.to_csv(output_path, index=False)